# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
import h5py
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
%matplotlib inline

# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
url = 'http://ufldl.stanford.edu/housenumbers/'
last_percent_reported = None
# 
def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 1% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  return filename

train_filename = maybe_download('train.tar.gz')
test_filename = maybe_download('test.tar.gz')
extra_filename = maybe_download('extra.tar.gz')

In [0]:
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = root
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)
extra_folders = maybe_extract(extra_filename)

train already present - Skipping extraction of train.tar.gz.
train
test already present - Skipping extraction of test.tar.gz.
test
extra already present - Skipping extraction of extra.tar.gz.
extra


In [0]:
import h5py

# The DigitStructFile is just a wrapper around the h5py data.  It basically references 
#    inf:              The input h5 matlab file
#    digitStructName   The h5 ref to all the file names
#    digitStructBbox   The h5 ref to all struc data
class DigitStructFile:
    def __init__(self, inf):
        self.inf = h5py.File(inf, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']
    def getName(self,n):
        # getName returns the 'name' string for for the n(th) digitStruct. 
        return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]].value])

    def bboxHelper(self,attr):
        # bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox.
        if (len(attr) > 1):
            attr = [self.inf[attr[j][0]][0][0] for j in range(len(attr))]
        else:
            attr = [attr[0][0]]
        return attr

    def getBbox(self,n):
        # getBbox returns a dict of data for the n(th) bbox.
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

    def getAllDigitStructure(self):
        # getAllDigitStructure returns all the digitStruct from the input file.
        result = []
        for i in range(len(self.digitStructName)):
            if (i % 1000 == 0):
                print(i)
            image_data = self.getDigitStructure(i)
            figures = []
            item = {"filename": image_data["name"]}
            for j in range(len(image_data["height"])):
                temp = [image_data["label"][j],
                        image_data["height"][j], 
                        image_data["left"][j], 
                        image_data["top"][j], 
                        image_data["width"][j]]
                figures.append(temp)
            item['boxes'] = figures
            result.append(item)
        return result

In [0]:
import os
train_folders = 'train'
test_folders = 'test'
extra_folders = 'extra'

In [0]:
fin = os.path.join(train_folders, 'digitStruct.mat')
dsf = DigitStructFile(fin)
train_data = dsf.getAllDigitStructure()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000


In [0]:
fin = os.path.join(test_folders, 'digitStruct.mat')
dsf = DigitStructFile(fin)
test_data = dsf.getAllDigitStructure()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [0]:
fin = os.path.join(extra_folders, 'digitStruct.mat')
dsf = DigitStructFile(fin)
extra_data = dsf.getAllDigitStructure()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [0]:
import cv2
def generate_dataset(data, folder):
    dataset_rgb = np.ndarray([len(data),32,32,3], dtype='float32')
    dataset_gray = np.ndarray([len(data), 32, 32, 1], dtype='float32')
    labels = np.ones([len(data),6], dtype=int) * 10
    for i in np.arange(len(data)):
        if (i % 1000 == 0):
            print(i)
        filename = data[i]['filename']
        fullname = os.path.join(folder, filename)
        im = cv2.cvtColor(cv2.imread(fullname), cv2.COLOR_BGR2RGB)
        boxes = data[i]['boxes']
        num_digit = len(boxes)
        labels[i,0] = num_digit
        top = np.ndarray([num_digit], dtype='float32')
        left = np.ndarray([num_digit], dtype='float32')
        height = np.ndarray([num_digit], dtype='float32')
        width = np.ndarray([num_digit], dtype='float32')  
        for j in np.arange(num_digit):
            if j < 5: 
                labels[i,j+1] = boxes[j][0]
                if boxes[j][0] == 10: labels[i,j+1] = 0
            else:
                print('#',i,'image has more than 5 digits.')
                continue
            top[j] = boxes[j][3]
            left[j] = boxes[j][2]
            height[j] = boxes[j][1]
            width[j] = boxes[j][4]
        ha = np.max([0, np.int16(np.min(top))])
        wa = np.max([0, np.int16(np.min(left))])
        hb = np.int16(np.max(height) + ha)
        wb = np.int16(np.sum(width)  + wa)
        h3 = (hb - ha) * .3
        w3 = (wb - wa) * .3
        ha = np.max([0,  np.int16(ha - h3)])
        hb = np.min([im.shape[0], np.int16(hb + h3)])
        wa = np.max([0,  np.int16(wa - w3)])
        wb = np.min([im.shape[1], np.int16(wb + w3)])
        im = im[ha:hb, wa:wb, :]
        im = cv2.resize(im, (32, 32))
        im_gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        im_gray = im_gray.reshape((32, 32, 1))
        dataset_rgb[i,:,:,:] = im[:,:,:]
    return dataset_rgb, dataset_gray, labels

In [0]:
# train_rgb, train_gray, train_labels = generate_dataset(train_data, 'train')
# test_rgb, test_gray, test_labels = generate_dataset(test_data, 'test')
extra_rgb, extra_gray, extra_labels = generate_dataset(extra_data, 'extra')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [0]:
import os

if not os.path.exists("data"):
  os.mkdir("data")

if not os.path.exists("data/rgb"):
  os.mkdir("data/rgb")

if not os.path.exists("data/gray"):
  os.mkdir("data/gray")

if not os.path.exists("data/labels"):
  os.mkdir("data/labels")

# np.save("data/rgb/train", train_rgb)
# np.save("data/gray/train", train_gray)
# np.save("data/labels/train", train_labels)

# np.save("data/rgb/test", test_rgb)
# np.save("data/gray/test", test_gray)
# np.save("data/labels/test", test_labels)

np.save("data/rgb/extra", extra_rgb)
np.save("data/gray/extra", extra_gray)
np.save("data/labels/extra", extra_labels)

In [0]:
import numpy as np
train_rgb = np.load("data/rgb/train.npy")
train_gray = np.load("data/gray/train.npy")
train_labels = np.load("data/labels/train.npy")

test_rgb = np.load("data/rgb/test.npy")
test_gray = np.load("data/gray/test.npy")
test_labels = np.load("data/labels/test.npy")

extra_rgb = np.load("data/rgb/extra.npy")
extra_gray = np.load("data/gray/extra.npy")
extra_labels = np.load("data/labels/extra.npy")

In [0]:
full_gray = np.concatenate([train_gray, extra_gray])
full_rgb = np.concatenate([train_rgb, extra_rgb])
full_labels = np.concatenate([train_labels, extra_labels])

import numpy as np
arr_rand = np.random.rand(len(train_labels) + len(extra_labels))
split = arr_rand < np.percentile(arr_rand, 75)

X_train_gray = full_gray[split]
X_valid_gray = full_gray[~split]
X_test_gray = test_gray

X_train = full_rgb[split]
X_valid = full_rgb[~split]
X_test = test_rgb

y_train = full_labels[split]
y_valid = full_labels[~split]
y_test = test_labels

In [0]:
import os

if not os.path.exists("derived_data"):
  os.mkdir("derived_data")

if not os.path.exists("derived_data/gray_train_valid_test"):
  os.mkdir("derived_data/gray_train_valid_test")

if not os.path.exists("derived_data/labels"):
  os.mkdir("derived_data/labels")

if not os.path.exists("derived_data/rgb_train_valid_test"):
  os.mkdir("derived_data/rgb_train_valid_test")

np.save('derived_data/gray_train_valid_test/X_train_gray.npy', X_train_gray)
np.save('derived_data/gray_train_valid_test/X_valid_gray.npy', X_valid_gray)
np.save('derived_data/gray_train_valid_test/X_test_gray.npy', X_test_gray)

np.save('derived_data/labels/y_train.npy', y_train)
np.save('derived_data/labels/y_valid.npy', y_valid)
np.save('derived_data/labels/y_test.npy', y_test)

np.save('derived_data/rgb_train_valid_test/X_train.npy', X_train)
np.save('derived_data/rgb_train_valid_test/X_valid.npy', X_valid)
np.save('derived_data/rgb_train_valid_test/X_test.npy', X_test)

In [0]:
import numpy as np

X_train = np.load('derived_data/rgb_train_valid_test/X_train.npy')
X_valid = np.load('derived_data/rgb_train_valid_test/X_valid.npy')
X_test = np.load('derived_data/rgb_train_valid_test/X_test.npy')

y_train = np.load('derived_data/labels/y_train.npy')
y_valid = np.load('derived_data/labels/y_valid.npy')
y_test = np.load('derived_data/labels/y_test.npy')


In [0]:
num = len(y_train)
reshaped_y_train = [(np.reshape(y_train[:num, 1], (num, 1))).astype('uint8'),
                    (np.reshape(y_train[:num, 2], (num, 1))).astype('uint8'),
                    (np.reshape(y_train[:num, 3], (num, 1))).astype('uint8'),
                    (np.reshape(y_train[:num, 4], (num, 1))).astype('uint8'),
                    (np.reshape(y_train[:num, 5], (num, 1))).astype('uint8')]

num = len(y_valid)
reshaped_y_valid = [(np.reshape(y_valid[:num, 1], (num, 1))).astype('uint8'),
                    (np.reshape(y_valid[:num, 2], (num, 1))).astype('uint8'),
                    (np.reshape(y_valid[:num, 3], (num, 1))).astype('uint8'),
                    (np.reshape(y_valid[:num, 4], (num, 1))).astype('uint8'),
                    (np.reshape(y_valid[:num, 5], (num, 1))).astype('uint8')]

num = len(y_test)
reshaped_y_test = [(np.reshape(y_test[:num, 1], (num, 1))).astype('uint8'),
                    (np.reshape(y_test[:num, 2], (num, 1))).astype('uint8'),
                    (np.reshape(y_test[:num, 3], (num, 1))).astype('uint8'),
                    (np.reshape(y_test[:num, 4], (num, 1))).astype('uint8'),
                    (np.reshape(y_test[:num, 5], (num, 1))).astype('uint8')]

In [0]:
rgb_data = {}
rgb_data['trainX'] = X_train
rgb_data['valdX'] = X_valid
rgb_data['testX'] = X_test
rgb_data['trainY'] = reshaped_y_train
rgb_data['valdY'] = reshaped_y_valid
rgb_data['testY'] = reshaped_y_test

gray_data = {}
gray_data['trainX'] = X_train_gray
gray_data['valdX'] = X_valid_gray
gray_data['testX'] = X_test_gray
gray_data['trainY'] = reshaped_y_train
gray_data['valdY'] = reshaped_y_valid
gray_data['testY'] = reshaped_y_test

In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16


def scratchVGG16_Model(X_train, X_valid, X_test, reshaped_y_train, reshaped_y_valid, reshaped_y_test):
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config = config)
    tf.keras.backend.set_session(sess)
    
    if tf.test.gpu_device_name():
      print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    else:
       print("Please install GPU version of TF")
    _,row, col,channel = X_train.shape
    digLen = 5 # including category 0
    numDigits = 11
    epochs = 50
    batch_size = 64
    
    vgg16Model = VGG16(include_top = False, weights = None)
    vgg16Model.summary()
    ptInput = tf.keras.Input(shape = (row,col,channel), name  = 'vgg16Scratch')
    vgg16 = vgg16Model(ptInput)

    vgg16 = Flatten()(vgg16)
    vgg16 = Dense(512, activation='relu')(vgg16)
    vgg16 = Dense(512, activation='relu')(vgg16)
    vgg16 = Dropout(0.5)(vgg16)
    
    d1 = Dense(11, activation='softmax')(vgg16)
    d2 = Dense(11, activation='softmax')(vgg16)
    d3 = Dense(11, activation='softmax')(vgg16)
    d4 = Dense(11, activation='softmax')(vgg16)
    d5 = Dense(11, activation='softmax')(vgg16)
    out = [d1, d2, d3, d4, d5]
    
    vgg16 = tf.keras.Model(inputs = ptInput, outputs = out)

    callback = []
    optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    modName = 'vgg16_scratch'
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='models/{}/vgg16.classifier.hdf5'.format(modName),
                                                   monitor='loss',
                                                   save_best_only=True,
                                                   verbose=2)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                                  factor = 0.1,
                                                  verbose = 1,
                                                  patience= 3,
                                                  cooldown= 0,
                                                  min_lr = 0.000001)
    es = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                       min_delta=0.00000001,
                                       patience=5,
                                       verbose=1,
                                       mode='auto')
    callback.append(es)
    callback.append(checkpointer)
    callback.append(reduce_lr)
    vgg16.summary()

    vgg16.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer= optim,
                  metrics=['accuracy'])

    vgg16History = vgg16.fit(x = X_train,
                             y = reshaped_y_train,
                             batch_size = batch_size,
                             epochs=1,
                             verbose=1,
                             shuffle = True,
                             validation_data = (X_valid, reshaped_y_valid),
                             callbacks = callback)
    pickle_file = 'models/{}/history.pickle'.format(modName)
    pickle.dump(vgg16History.history, open(pickle_file, 'wb'))
    vgg16.save("models/{}/model.h5".format(modName))

In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16



config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
tf.keras.backend.set_session(sess)

if tf.test.gpu_device_name():
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")
_,row, col,channel = X_train.shape
digLen = 5 # including category 0
numDigits = 11
epochs = 50
batch_size = 64

vgg16Model = VGG16(include_top = False, weights = None)
vgg16Model.summary()
ptInput = tf.keras.Input(shape = (row,col,channel), name  = 'vgg16Scratch')
vgg16 = vgg16Model(ptInput)

vgg16 = Flatten()(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
vgg16 = Dropout(0.5)(vgg16)

d1 = Dense(11, activation='softmax')(vgg16)
d2 = Dense(11, activation='softmax')(vgg16)
d3 = Dense(11, activation='softmax')(vgg16)
d4 = Dense(11, activation='softmax')(vgg16)
d5 = Dense(11, activation='softmax')(vgg16)
out = [d1, d2, d3, d4, d5]

vgg16 = tf.keras.Model(inputs = ptInput, outputs = out)

callback = []
optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
modName = 'vgg16_scratch'
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='models/{}/vgg16.classifier.hdf5'.format(modName),
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 3,
                                              cooldown= 0,
                                              min_lr = 0.000001)
es = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                   min_delta=0.00000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)
vgg16.summary()

vgg16.compile(loss = 'sparse_categorical_crossentropy',
              optimizer= optim,
              metrics=['accuracy'])

vgg16History = vgg16.fit(x = X_train,
                         y = reshaped_y_train,
                         batch_size = batch_size,
                         epochs=epochs,
                         verbose=1,
                         shuffle = True,
                         validation_data = (X_valid, reshaped_y_valid),
                         callbacks = callback)
pickle_file = 'models/{}/history.pickle'.format(modName)
pickle.dump(vgg16History.history, open(pickle_file, 'wb'))
vgg16.save("models/{}/model.h5".format(modName))

Default GPU Device: /device:GPU:0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, 

In [0]:
vgg16 = tf.keras.models.load_model("models/vgg16_scratch/model.h5")

In [0]:
vgg16.to_json()

'{"class_name": "Model", "config": {"name": "model_13", "layers": [{"name": "vgg16Scratch", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 32, 32, 3], "dtype": "float32", "sparse": false, "name": "vgg16Scratch"}, "inbound_nodes": []}, {"name": "vgg16", "class_name": "Model", "config": {"name": "vgg16", "layers": [{"name": "input_14", "class_name": "InputLayer", "config": {"batch_input_shape": [null, null, null, 3], "dtype": "float32", "sparse": false, "name": "input_14"}, "inbound_nodes": []}, {"name": "block1_conv1", "class_name": "Conv2D", "config": {"name": "block1_conv1", "trainable": true, "dtype": "float32", "filters": 64, "kernel_size": [3, 3], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel

In [0]:
with open('models/vgg16_scratch/history.pickle', "rb") as f:
  data = pickle.load(f)

In [0]:
data

{'dense_93_acc': [0.43149376,
  0.7894252,
  0.91079426,
  0.9295822,
  0.9389252,
  0.9439304,
  0.9478102,
  0.9505192,
  0.9529228,
  0.95335263,
  0.95243645,
  0.9547439,
  0.9561409,
  0.95754343,
  0.95827866,
  0.9583861,
  0.9593702,
  0.95724934,
  0.956418,
  0.95927405,
  0.9584257,
  0.9603825,
  0.96116865,
  0.9627749,
  0.96111214,
  0.9599301,
  0.9628767,
  0.96263915,
  0.9625543],
 'dense_93_loss': [1.6001005,
  0.63884205,
  0.3432035,
  0.2858938,
  0.2482164,
  0.23120044,
  0.21880156,
  0.20738517,
  0.19578572,
  0.196203,
  0.19945613,
  0.19315474,
  0.18544273,
  0.18337029,
  0.17896126,
  0.18048541,
  0.17360368,
  0.18148269,
  0.18352747,
  0.17532897,
  0.1791029,
  0.17263381,
  0.1684972,
  0.16472635,
  0.16747439,
  0.17354739,
  0.1612758,
  0.16401517,
  0.16448031],
 'dense_94_acc': [0.27597615,
  0.7112083,
  0.86746675,
  0.89996946,
  0.91315264,
  0.92166436,
  0.92881864,
  0.93194056,
  0.93562233,
  0.9366573,
  0.9369005,
  0.93942857,


In [0]:
import os
import pickle
if not os.path.exists("models"):
  os.mkdir("models")

if not os.path.exists("models/vgg16_scratch"):
  os.mkdir("models/vgg16_scratch")

scratchVGG16_Model(X_train, X_valid, X_test, reshaped_y_train, reshaped_y_valid, reshaped_y_test)

In [0]:
import os
import pickle

modName = 'vgg16_pretrained'

if not os.path.exists("models"):
  os.mkdir("models")

if not os.path.exists("models/{}".format(modName)):
  os.mkdir("models/{}".format(modName))

In [0]:
import tensorflow as tf
import pickle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.vgg16 import VGG16

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
tf.keras.backend.set_session(sess)

if tf.test.gpu_device_name():
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")
_,row, col,channel = X_train.shape

digLen = 5
numDigits = 11
epochs = 50
batch_size = 64
optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

ptInput = tf.keras.Input(shape = (row,col,channel), name  = 'inputVGGPreTrain')
pt_vgg16 = VGG16(include_top = False, weights='imagenet')(ptInput)
pt_vgg16 = Flatten(name = 'flatten')(pt_vgg16)
pt_vgg16 = Dense(512, activation='relu', name = 'FC1_512')(pt_vgg16)
pt_vgg16 = Dense(512, activation='relu', name = 'FC2_512')(pt_vgg16)

d1 = Dense(11, activation='softmax')(pt_vgg16)
d2 = Dense(11, activation='softmax')(pt_vgg16)
d3 = Dense(11, activation='softmax')(pt_vgg16)
d4 = Dense(11, activation='softmax')(pt_vgg16)
d5 = Dense(11, activation='softmax')(pt_vgg16)
out = [d1, d2, d3, d4, d5]

vggPreTrain = tf.keras.Model(inputs=ptInput, outputs=out)

vggPreTrain.compile(loss = 'sparse_categorical_crossentropy', #ceLoss ,
                    optimizer= optim,
                    metrics=  ['accuracy']) #[])
vggPreTrain.summary()

callback = []
modName = 'vgg16_pretrained'

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='models/{}/VGGPreTrained.classifier.hdf5'.format(modName),
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 4,
                                              cooldown= 1,
                                              min_lr = 0.0001)
es = tf.keras.callbacks.EarlyStopping(monitor= 'loss',
                                   min_delta=0.000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)

vgg_pretrained_history = vggPreTrain.fit(x = X_train,
                                         y = reshaped_y_train,
                                         batch_size = batch_size,
                                         epochs=epochs,
                                         verbose=1,
                                         shuffle = True,
                                         validation_data = (X_valid, reshaped_y_valid),
                                         callbacks= callback)
pickle_file = 'models/{}/history.pickle'.format(modName)
pickle.dump(vgg_pretrained_history.history, open(pickle_file, 'wb'))
vggPreTrain.save("models/{}/model.h5".format(modName))

Default GPU Device: /device:GPU:0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 512)          0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_512 (Dense)                 (None, 512)          262656      flatten[0][0]                    
___________________________________________________________________________

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.350230318000058
GPU (s):
0.1842791589999706
GPU speedup over CPU: 45x
